Data Preproccessing
1. Collect the data (Hepatatis.csv)
2. Import neccessary Libraries (pandas, scikit-learn, matplot lib etc)
3. Import the dataset (using pd.read_csv)
4. Find and fix the missing values (simple imputer, column trnasformer)
5. Encoding (One-Hot encoding for X i.e features, and Label Encoding for y i.e target)
6. Split the the X, y (after encoding) into X_train, X_test, y_train, y_test
7. Feature Scaling (Donno shld look to it)

In [296]:
#1 collected dataset is Hepatatis.csv
#2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [297]:
#3
df = pd.read_csv('/kaggle/input/hepatatis/hepatitis.csv')
rows, cols = df.shape

# Print the result
print("Number of rows:", rows)
print("Number of columns:", cols)

Number of rows: 155
Number of columns: 20


In [298]:
dfcopy = df
df.head()

,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,30,male,False,False,False,False,False,False,False,False,False,False,False,1.0,85.0,18.0,4.0,NaN,False,live
1,50,female,False,False,True,False,False,False,False,False,False,False,False,0.9,135.0,42.0,3.5,NaN,False,live
2,78,female,True,False,True,False,False,True,False,False,False,False,False,0.7,96.0,32.0,4.0,NaN,False,live
3,31,female,NaN,True,False,False,False,True,False,False,False,False,False,0.7,46.0,52.0,4.0,80.0,False,live
4,34,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,200.0,4.0,NaN,False,live


In [299]:
Y = df['class']

In [300]:
X = df.drop(columns=['class'])

In [301]:
X.isnull().sum()

age                 0
sex                 0
steroid             1
antivirals          0
fatigue             1
malaise             1
anorexia            1
liver_big          10
liver_firm         11
spleen_palpable     5
spiders             5
ascites             5
varices             5
bilirubin           6
alk_phosphate      29
sgot                4
albumin            16
protime            67
histology           0
dtype: int64

In [302]:
# on x,get the missing values, if numerical replace with mean, if categorical replace with most_frequent
# Then do the OneHotEncoder on all the categorical values

In [303]:
num_cols = X.select_dtypes(include=['int', 'float']).columns
str_cols = X.select_dtypes(include=['object']).columns

In [304]:
num_cols

Index(['age', 'bilirubin', 'alk_phosphate', 'sgot', 'albumin', 'protime'], dtype='object')

In [305]:
str_cols

Index(['sex', 'steroid', 'fatigue', 'malaise', 'anorexia', 'liver_big',
       'liver_firm', 'spleen_palpable', 'spiders', 'ascites', 'varices'],
      dtype='object')

In [306]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [307]:
print(X_train.shape)
print(X_test.shape)

(124, 19)
(31, 19)


In [308]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder();
le.fit(Y_train)

LabelEncoder()

In [309]:
Y_train_trans = le.transform(Y_train)
Y_test_trans = le.transform(Y_test)

In [320]:
# On all numerical apply the simple
# On all cate first apply the simple, then the oneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [332]:
# handle_unknown = "ignore" is becuse we are fitting using the X_train, but say in the X_test has the new category which is not seen in X_train
# it gives out the error
# Scaling has to be applied on the numerical data
# Actually using the standardScaler doesnt make much diffrerece on tree type, like Decisiontree


#Standard scaler formula is
'''
xi' = (xi - xmean)/sd
with this the new cols will have mean=0, and SD =1, values lie in [-1,1]

There is one more feature scaling concept called MinMaxScaler
xi' = (xi-xmin)/(xmax-xmin)
values lie in [0,1]

.preprocessing import MinMaxScaler
'''


NumericalPipeline = Pipeline(
    [
        ('simpleImputer', SimpleImputer(strategy="mean")),
        ('standDardScalar', StandardScaler()) 
    ]
)

categoryPipeline = Pipeline(
    [
        ('simpleCat', SimpleImputer(strategy="most_frequent")),
        ('oneHot', OneHotEncoder(handle_unknown="ignore"))
    ]
)

transformer = ColumnTransformer(
    transformers= [
        ('numerical', NumericalPipeline, ['age', 'bilirubin', 'alk_phosphate', 'sgot', 'albumin', 'protime']),
        ('category', categoryPipeline, ['sex', 'steroid', 'fatigue', 'malaise', 'anorexia', 'liver_big',
       'liver_firm', 'spleen_palpable', 'spiders', 'ascites', 'varices'])
    ],
    remainder="passthrough"
)

In [323]:
transformer.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical',
                                 Pipeline(steps=[('simpleImputer',
                                                  SimpleImputer()),
                                                 ('standDardScalar',
                                                  StandardScaler())]),
                                 ['age', 'bilirubin', 'alk_phosphate', 'sgot',
                                  'albumin', 'protime']),
                                ('category',
                                 Pipeline(steps=[('simpleCat',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('oneHot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['sex', 'steroid', 'fatigue', 'malaise',
                                  'anorexia', 'liver_big', 'liver_firm',
                                  'spleen_palpable', 'spiders', 'ascites',
                                  'varices'])])

In [324]:
X_train_trans = transformer.transform(X_train)
X_test_trans = transformer.transform(X_test)

In [325]:
X_train_trans

array([[ 1.61269736, -0.06385753,  0.67463293, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.3001236 ,  0.85938391,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.00871728, -0.43315411, -0.58675616, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.99501559, -0.15618168, -0.50266355, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47197861,  0.12079076,  0.61156348, ...,  0.        ,
         0.        ,  1.        ],
       [-0.39476839, -0.6178024 ,  1.15816541, ...,  0.        ,
         0.        ,  0.        ]])

In [330]:
# Unable to read them, so convert back to DataFrame (optional)
X_train_trans1 = pd.DataFrame(X_train_trans, columns=transformer.get_feature_names_out())
X_train_trans1

,numerical__age,numerical__bilirubin,numerical__alk_phosphate,numerical__sgot,numerical__albumin,numerical__protime,category__sex_female,category__sex_male,category__steroid_False,category__steroid_True,...,category__spleen_palpable_False,category__spleen_palpable_True,category__spiders_False,category__spiders_True,category__ascites_False,category__ascites_True,category__varices_False,category__varices_True,remainder__antivirals,remainder__histology
0,1.612697,-0.063858,0.674633,0.713854,1.091608e-01,-2.985414e-01,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.300124,0.859384,0.000000,5.929726,7.022970e-16,-8.344032e-16,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,-0.008717,-0.433154,-0.586756,-0.303877,1.091608e-01,-6.508369e-01,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.840595,-0.617802,-0.712895,-0.356884,2.673040e-01,-2.471030e+00,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.068493,-0.802451,-0.986196,-0.219066,-4.898242e-02,-2.001303e+00,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0.686175,-0.340830,0.632587,-0.081248,1.091608e-01,-6.367781e-02,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
120,-0.780819,6.121860,0.000000,0.130779,-2.579274e+00,-8.344032e-16,1.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
121,0.995016,-0.156182,-0.502664,0.035367,-1.155985e+00,1.711858e-01,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
122,-0.471979,0.120791,0.611563,-0.325080,-1.946701e+00,-8.344032e-16,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [331]:
X_train_trans1.describe()

,numerical__age,numerical__bilirubin,numerical__alk_phosphate,numerical__sgot,numerical__albumin,numerical__protime,category__sex_female,category__sex_male,category__steroid_False,category__steroid_True,...,category__spleen_palpable_False,category__spleen_palpable_True,category__spiders_False,category__spiders_True,category__ascites_False,category__ascites_True,category__varices_False,category__varices_True,remainder__antivirals,remainder__histology
count,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,1.240000e+02,124.000000,124.000000,124.000000,124.000000,...,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000
mean,2.820325e-16,2.014518e-16,5.551115e-17,7.879002e-17,3.814153e-16,-8.058070e-16,0.887097,0.112903,0.508065,0.491935,...,0.806452,0.193548,0.685484,0.314516,0.862903,0.137097,0.887097,0.112903,0.153226,0.491935
std,1.004057e+00,1.004057e+00,1.004057e+00,1.004057e+00,1.004057e+00,1.004057e+00,0.317758,0.317758,0.501963,0.501963,...,0.396682,0.396682,0.466207,0.466207,0.345345,0.345345,0.317758,0.317758,0.361666,0.501963
min,-2.633865e+00,-9.870990e-01,-1.658937e+00,-7.915397e-01,-2.737417e+00,-3.704064e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.036093e-01,-5.254783e-01,-6.498256e-01,-6.113165e-01,-5.234121e-01,-3.572573e-01,1.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,-2.017428e-01,-3.408300e-01,0.000000e+00,-3.038769e-01,2.673040e-01,-8.344032e-16,1.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,7.054773e-01,1.207908e-01,3.435183e-01,2.367928e-01,5.835905e-01,1.711858e-01,1.000000,0.000000,1.000000,1.000000,...,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,2.848061e+00,6.121860e+00,3.912198e+00,5.929726e+00,4.062742e+00,2.167527e+00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
